In [11]:
import random
from tqdm.notebook import tqdm

In [12]:
class Process:
    def __init__(self, pid, length):
        self.pid = pid
        self.length = length

    def __repr__(self):
        return f'Process(pid={self.pid}, length={self.length})'

In [13]:
class Server:
    def __init__(self, core_speed, num_cores):
        self.core_speed = core_speed
        self.num_cores = num_cores
        self.workload = []

    def __repr__(self):
        return f'Server(core_speed={self.core_speed}, num_cores={self.num_cores}, workload={self.workload})'

In [14]:
def evaluate_fitness(member):
    processes, servers = member
    fitness = 0
    for server in tqdm(servers, desc="Evaluating fitness"):
        execution_time = 0
        for process in processes:
            if process in server.workload:
                execution_time += process.length / (server.core_speed * server.num_cores)
        fitness += execution_time
    return fitness

In [15]:
def get_fittest_indexes(fitnesses, population_size):
    return sorted(range(len(fitnesses)), key=lambda x: fitnesses[x])[:int(population_size/2)]

In [16]:
def create_offspring(fittest, population_size):
    offspring = []
    while tqdm(len(offspring) < population_size, desc="Breeding"):
        parent1 = random.choice(fittest)
        parent2 = random.choice(fittest)
        child = breed(parent1, parent2)
        offspring.append(child)
    return offspring

In [17]:
def mutate(member):
    processes, servers = member
    # Choose a random process and assign it to a random server
    process = random.choice(processes)
    server = random.choice(servers)
    process_index = processes.index(process)
    processes[process_index] = None
    server.workload.append(process)
    return [processes, servers]

In [18]:
def mutate_offspring(offspring):
    for i in range(len(offspring)):
        if random.random() < 0.1:
            offspring[i] = mutate(offspring[i])
    return offspring

In [19]:
def breed(parent1, parent2):
    # Combine the process lists of the parents and shuffle them to create the child
    child_processes = parent1[0] + parent2[0]
    random.shuffle(child_processes)
    return [child_processes, parent1[1]]

In [20]:
def assign_processes(processes, servers, population_size=100, n_iter=100):
    # Initialize the population with randomly assigned sets of processes to servers
    population = []
    for _ in range(population_size):
        population.append([random.sample(processes, k=len(processes)), servers])
    
    for i in tqdm(range(n_iter), desc="Assign process"):
        # Evaluate the fitness of each member of the population
        fitnesses = []
        for member in population:
            fitnesses.append(evaluate_fitness(member))
        
        # Select the fittest members of the population for reproduction
        fittest_indexes = get_fittest_indexes(fitnesses, population_size)
        fittest = [population[i] for i in fittest_indexes]
        
        # Breed new members by combining the process lists of the fittest members
        offspring = create_offspring(fittest, population_size)
        
        # Add some random mutations to the process lists of the new members
        offspring = mutate_offspring(offspring)
        
        # Replace the current population with the offspring
        population = offspring
    
    # Return the fittest member of the final population
    fitnesses = []
    for member in population:
        fitnesses.append(evaluate_fitness(member))
    fittest_index = min(range(len(fitnesses)), key=lambda x: fitnesses[x])
    return population[fittest_index]